In [1]:
import os 
import pandas as pd 
# scrape edilenler ile txt dosyası arasındaki farklı linkleri gösterir. (df,txt_dosyasi,file_path'in sonlarını düzenle)

def farkli_(json,txt):
    df = pd.read_json(f'/Users/icy/Desktop/scrapping_scrapy_football/football/football/spiders/{json}.json')
    txt_dosyasi = pd.read_csv(f'/Users/icy/Desktop/scrapping_scrapy_football/football/football/spiders/sofa_temiz_urls/{txt}.txt',header=None, names=['a'])  
    txt_url = set(txt_dosyasi['a'])
    df_url = set(df['match_link'])
    
    unique_to_x = txt_url - df_url
    unique_to_y = df_url - txt_url

    directory = '/Users/icy/Desktop/scrapping_scrapy_football/football/football/spiders/sofa_temiz_urls'
    file_path = os.path.join(directory, f'{txt}_unscraped.txt')
    matches_liste = list(unique_to_x)
    with open(file_path, 'w') as file:
        for url in matches_liste:
            file.write(url + '\n')
    
    print(f"Unique to x: {len(unique_to_x)} URLs\n"
          f"Unique to y: {len(unique_to_y)} URLs\n"
          f"List unique to y: {list(unique_to_y)}\n"
          f"List unique to x: {list(unique_to_x)}")

# Örnek kullanım:
# import pandas as pd
# x = pd.DataFrame({'a': ['url1', 'url2', 'url3']})
# y = pd.DataFrame({'match_link': ['url3', 'url4']})
# farklı_urls(x, y)
farkli_('firsthalf_goal','son_eksik_maclar')  #-> json,txt iki değer alır.


In [ ]:
# ham veriye uyguladıklarım 

gerekli_orglar = ['Serie A','Super Lig','LaLiga','Bundesliga','Premier League','Ligue 1','Eredivisie',
                'Liga Portugal Betclic',
                'UEFA Champions League, Group E', 'UEFA Champions League, Group F','UEFA Champions League, Group G', 
                'UEFA Champions League, Group H','UEFA Champions League, Group A', 'UEFA Champions League, Group B',
                'UEFA Champions League, Group C', 'UEFA Champions League, Group D','UEFA Champions League, Knockout stage']
son_matches = son_matches[son_matches.Organizasyon.isin(gerekli_orglar)]
son_matches = son_matches[~son_matches.Away_goal.isnull()]
son_matches = son_matches[~(son_matches.Home_goal == 'Postponed')]
son_matches = son_matches[~(son_matches.Home_goal == 'Canceled')]
son_matches = son_matches[~son_matches.ulke.isin([ 'Russia', 'Austria','Singapore', 'Ghana', 'Bangladesh', 'Congo Republic'])]
son_matches = son_matches[['date', 'hour', 'ulke', 'Organizasyon', 'Org_detay', 'Home_team','Away_team', 'Home_goal', 'Away_goal',
'Ball possession_home','Ball possession_away','Total shots_home', 'Total shots_away','Corner kicks_home', 'Corner kicks_away',
'Fouls_home', 'Fouls_away','Free kicks_home', 'Free kicks_away','Passes_home', 'Passes_away',
'Yellow cards_home', 'Yellow cards_away', 'Red cards_home','Red cards_away','referee','red_card_avg','yellow_card_avg', 
'home_win_odd','draw_odd', 'away_win_odd','match_link']]
son_matches.sort_values(by=['date','hour','Organizasyon'],ascending= False, inplace= True , ignore_index= True )


In [ ]:
# DataFramei oluşturma
year_list = ['2024_1','2023_2','2023_1','2022_2','2022_1','2021_2','2021_1','2020_2','2020_1',
             '2019_2','2019_1','2018_2','2018_1','2017_2','2017_1']
matches = pd.DataFrame()
for year in year_list : 
    df = pd.read_json(f'/Users/icy/Desktop/scrapping_scrapy_football/football/football/spiders/matches_sofa{year}.json')
    matches = pd.concat([matches,df], axis= 0)

In [ ]:
# season_period, season_round columunu oluşturma /int /oluşturduktan sonra last_df_.csv oluşturur o an olanı yedekler.
df_leagues = df[~(df.ulke == 'Europe')]
df_leagues.loc[ : , 'season_round'] = df_leagues.org_detay.apply(lambda x: x[-2:]).astype(int)
df_leagues = df_leagues[['match_link','season_round']]
df_leagues.loc[df_leagues.season_round <= 9 , 'season_period'] = 1
df_leagues.loc[(df_leagues.season_round > 9) & (df_leagues.season_round <= 18)  , 'season_period'] = 2
df_leagues.loc[(df_leagues.season_round > 18) & (df_leagues.season_round <= 27)  , 'season_period'] = 3
df_leagues.loc[(df_leagues.season_round >= 28)  , 'season_period'] = 4
merged_df = pd.merge(df,df_leagues, on= 'match_link', how= 'left')
merged_df = merged_df.drop_duplicates()
merged_df = merged_df.fillna(0)
merged_df.season_period = merged_df.season_period.astype(int)
merged_df.season_round = merged_df.season_round.astype(int)
merged_df.reset_index(drop= True, inplace= True)
yedek_df = pd.read_csv('/Users/icy/Football_project/Footbal_project/csvler/last_df_.csv')
yedek_df.to_csv('/Users/icy/Football_project/Footbal_project/csvler/last_df_yedek.csv')
merged_df.to_csv('/Users/icy/Football_project/Footbal_project/csvler/last_df_.csv')